# Resumo
Este projeto é dedicado à criação de um corretor ortográfico para palavras da língua portuguesa brasileira. O script utilizado foi inspirado no apresentado durante o curso "Corretor Ortográfico em Python: Aplicando técnicas de NLP" que frequentei na plataforma Alura.

# Base de Dados
Uma pequena parte do conjunto de palavras reunido pelo Núcleo Interinstitucional de Linguística Computacional da Universidade de São Paulo (NILC/USP) será utilizado como corpus textual para o corretor ortográfico. A compilação foi realizada para representar o léxico português mais ouvido por crianças e foram utilizadas 5 milhões de arquivos de legendas de filmes e séries dos gêneros Família e Animação do site Open Subtitle.

**Disponível em:** http://www.nilc.icmc.usp.br/leg2kids/

**Arquivo utilizado:** 'Legendas pré-processadas'. 

Legendas extraídas de seu formato original e processadas com o intuito de remover as marcações de tempo, remover os travessões que indicam fala e remover marketing, comumente encontrando em legendas produzidas por terceiros. Cada legenda está salva em um arquivo único. Cada trecho de fala da legenda está em uma linha do arquivo. Todos os arquivos possuem codificação utf-8. Conteúdo limpo e "cru", indicado para propósitos gerais. Conteúdo em formato zip.

# Importando o Corpus Textual

In [112]:
# acessando arquivo de texto via url do dropbox
url = 'https://www.dropbox.com/s/ufyw476gan5sr6j/corpus_raw.txt?dl=1'
import urllib.request

u = urllib.request.urlopen(url)
data = u.read()
u.close()

with open('corpus_palavras.txt', 'wb') as f :
     f.write(data)

# lendo o arquivo
with open("corpus_palavras.txt", "r") as f:
     corpus = f.read()
  
print(corpus[:200])

 ^ MINHA BELA DAMA
 ^ Freddy, vá chamar um táxi.
 ^ Você quer que eu pegue uma pneumonia?
 ^ Não fique aí parado.
 ^ Chame um táxi.
 ^ Está bem, vou chamar.
 ^ Olhe por onde anda,
 ^ Olhe por onde and


# Avaliando o Corpus Textual

Processo elencados:
* Tokenização - 
* Normalização - 
* Contagemn de palavras -

## Tokenização

Uma tentativa de tokenização poderia ser realizada com o método split:

In [113]:
tokens_split = corpus.split()

In [114]:
print(f'Com o método split teríamos {len(corpus.split())} tokens.')

Com o método split teríamos 661079 tokens.


Tokens estão separados, todavia, elementos de pontuação aparecem junto com as palavras. Como o token abaixo:

In [115]:
tokens_split[5]

'Freddy,'

Para resolver esse problema, será utilizada a bibliteca 'Natural Language Toolkit' no processo de tokenização.

In [116]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Aplicando o método 'word_tokenize' no corpus textual.

In [117]:
tokens = nltk.tokenize.word_tokenize(corpus)
tokens[:10]

['^', 'MINHA', 'BELA', 'DAMA', '^', 'Freddy', ',', 'vá', 'chamar', 'um']

O método resultou na tokenização de palavras, mas ainda apresenta os caracteres de pontuação como tokens.

Para refinar a contagem de palavras do corpus será criada a função 'separa_apenas_palavras'.

In [118]:
def separa_apenas_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
           lista_palavras.append(token)
    return lista_palavras

Aplicando a função 'separa_apenas_palavra'

In [119]:
corpus_palavras = separa_apenas_palavras(tokens)

# visualização das 10 primeiras palavras do corpus tratado
corpus_palavras[:10]

['MINHA',
 'BELA',
 'DAMA',
 'Freddy',
 'vá',
 'chamar',
 'um',
 'táxi',
 'Você',
 'quer']

Após sucesso da tokenização, será calculado o total de palavras do corpus.

In [120]:
print(f'O número total de palavras do corpus é {len(corpus_palavras)}.')

O número total de palavras do corpus é 525378.


## Contagem de Palavras do Corpus

Para descobrir quantas palavras distintas possuímos no corpus serão realizados: 
* Normalização do texto - aplicar uma transformação para que palavras iguais sejam reconhecidas da mesma forma, não importando se foram utilizadas letras em caixa alta.
* Removeção de palavras repetidas. 

## Normalização

Como é possível observer no exemplo abaixo, a lista atual não transforma o texto e armazena palavras com letras maíuscula.

In [121]:
print(corpus_palavras[:5])

['MINHA', 'BELA', 'DAMA', 'Freddy', 'vá']


Portanto, uma função chamada 'normalizacao' será criada visando a normalização de todas as palavras de um corpus.

In [122]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

Assim, a normalização da variável 'corpus_palavras' será realizada.

In [123]:
corpus_normalizado = normalizacao(corpus_palavras)

print(corpus_normalizado[:10])

['minha', 'bela', 'dama', 'freddy', 'vá', 'chamar', 'um', 'táxi', 'você', 'quer']


# Contagem de palavras não repetidas

Para remover das palavras repetidas da contagem do corpus será utilizada a ferramenta de implementação de conjuntos em python (**set**). Desta forma, será evidenciado a quantidade de palavras que o corretor ortográfico aqui construído saberá corrigir. 

In [124]:
corpus_sem_repeticao = set(corpus_normalizado)
palavras_repetidas = len(corpus_normalizado) - len(corpus_sem_repeticao)

print(f'O corpus tem o total de {len(corpus_normalizado)} palavras.')
print(f'O corpus possui {palavras_repetidas} palavras repetidas.')
print(f'O corretor saberá corrigir {len(corpus_sem_repeticao)} palavras.')

O corpus tem o total de 525378 palavras.
O corpus possui 499747 palavras repetidas.
O corretor saberá corrigir 25631 palavras.


# Criando função de correção mediante a adição de uma letra

Nesta seção, será criado um algoritmo para a sugestão de correção em palavras digitadas faltando caracteres (ex:. 'lgica').

Para esse processo, será necessário um mecanismo fatie a string de input e teste a inserção de diferentes letras em diferentes posições - gerando palavras - para realizar uma sugestão ao usuário. Como no exemplo a seguir:

In [125]:
# exemplo de fatiamento de palavra
lista = [1, 2, 3]
print(f'lista = {lista}')
print(lista[0])
print(lista[1:2])
print(lista[1:3])
print()

lista = "lgica"
print(f'lista = {lista}')
print(f'1º fatiamento: {(lista[:0], lista[0:])}')
print(f'2º fatiamento: {(lista[:1], lista[1:])}')
print(f'3º fatiamento: {(lista[:2], lista[2:])}')
print(f'4º fatiamento: {(lista[:3], lista[3:])}')
print(f'5º fatiamento: {(lista[:4], lista[4:])}')
print(f'6º fatiamento: {(lista[:5], lista[5:])}')

lista = [1, 2, 3]
1
[2]
[2, 3]

lista = lgica
1º fatiamento: ('', 'lgica')
2º fatiamento: ('l', 'gica')
3º fatiamento: ('lg', 'ica')
4º fatiamento: ('lgi', 'ca')
5º fatiamento: ('lgic', 'a')
6º fatiamento: ('lgica', '')


Portanto, no processo de correção aqui desenvolvido, a palavra digitada equivocadamente será fatiada em duas partes de todas as formas demonstradas no exemplo anterior. Em seguida, cada letra do alfabeto (mais letras acentuadas e ç) será posicionada entre as partes fatiadas com o objetivo de formar um escopo de palavras possíveis com a adição de apenas uma letra à palavra digitada.

In [126]:
# função que insere uma letra entre duas fatias de uma palavra
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

# função que retorna possíveis palavras adicionando uma letra na digitação
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

Para exemplificar o funcionamento, será observado o comportamento do corretor ao receber a string "tste". O primeiro passo é o fatiamento da palavra. Depois o gerador retorna possíveis palavras formadas pela inserção da variável 'letras' (todas as letras do alfabeto, vogais acentuadas e o 'ç') entre as fatias da string.

In [127]:
palavra_exemplo = 'tste'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas)

['atste', 'btste', 'ctste', 'dtste', 'etste', 'ftste', 'gtste', 'htste', 'itste', 'jtste', 'ktste', 'ltste', 'mtste', 'ntste', 'otste', 'ptste', 'qtste', 'rtste', 'stste', 'ttste', 'utste', 'vtste', 'wtste', 'xtste', 'ytste', 'ztste', 'àtste', 'átste', 'âtste', 'ãtste', 'ètste', 'étste', 'êtste', 'ìtste', 'ítste', 'îtste', 'òtste', 'ótste', 'ôtste', 'õtste', 'ùtste', 'útste', 'ûtste', 'çtste', 'taste', 'tbste', 'tcste', 'tdste', 'teste', 'tfste', 'tgste', 'thste', 'tiste', 'tjste', 'tkste', 'tlste', 'tmste', 'tnste', 'toste', 'tpste', 'tqste', 'trste', 'tsste', 'ttste', 'tuste', 'tvste', 'twste', 'txste', 'tyste', 'tzste', 'tàste', 'táste', 'tâste', 'tãste', 'tèste', 'téste', 'têste', 'tìste', 'tíste', 'tîste', 'tòste', 'tóste', 'tôste', 'tõste', 'tùste', 'túste', 'tûste', 'tçste', 'tsate', 'tsbte', 'tscte', 'tsdte', 'tsete', 'tsfte', 'tsgte', 'tshte', 'tsite', 'tsjte', 'tskte', 'tslte', 'tsmte', 'tsnte', 'tsote', 'tspte', 'tsqte', 'tsrte', 'tsste', 'tstte', 'tsute', 'tsvte', 'tswte', 

A aplicação do corretor ocorrerá com a avaliação das palavras geradas (escopo de palavras possíveis). Por fim, a palavra com a maior maior frequência no corpus será retornada como sugestão. 

In [128]:
# função corretor que recebe a palavra errada e sugere a palavra correta
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key = probabilidade)
    return palavra_correta

Após gerar as paravras derivadas da errada, o corretor deverá apresentar a mais provável correção. A escolhida será, entre as derivadas, aquela mais frequente no corpus. Para tal, a função *probabilidade* irá retornar a taxa de *frequencia* de cada sugestão no corpus.

In [129]:
# número de palavras no corpus
total_palavras = len(corpus_normalizado)

# frequencia de cada palavra no corpus
frequencia = nltk.FreqDist(corpus_normalizado)
frequencia.most_common(10)

[('que', 17020),
 ('o', 15103),
 ('não', 13720),
 ('de', 12541),
 ('a', 11908),
 ('você', 11116),
 ('é', 10584),
 ('eu', 9633),
 ('e', 9383),
 ('um', 7025)]

In [130]:
# função que define a probabilidade das palavras geradas a ser a correta
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

In [131]:
probabilidade("de")

0.023870432336336886

In [132]:
# exemplos de uso da função com o corpus selecionado
print('USO DO CORRETOR COM PALAVRA DIGITADA FALTANDO UMA LETRA')
print(f'Input = "crretor" -  Sugestão = "{corretor("crretor")}"')
print(f'Input = "ága" -  Sugestão = "{corretor("ága")}"')
print(f'Input = "luzs" -  Sugestão = "{corretor("luzs")}"')
print(f'Input = "prbabilidade" -  Sugestão = "{corretor("prbabilidade")}"')
print()
print('USO DO CORRETOR COM PALAVRA DIGITADA FALTANDO DUAS LETRAS')
print(f'Input = "crrtor" -  Sugestão = "{corretor("crrtor")}"')
print(f'Input = "prbbilidade" -  Sugestão = "{corretor("prbbilidade")}"')
print()
print('USO DO CORRETOR COM PALAVRA DIGITADA COM LETRAS ERRADAS')
print(f'Input = "corrretor" -  Sugestão = "{corretor("corrretor")}"')
print(f'Input = "probabillidade" -  Sugestão = "{corretor("probabillidade")}"')

USO DO CORRETOR COM PALAVRA DIGITADA FALTANDO UMA LETRA
Input = "crretor" -  Sugestão = "corretor"
Input = "ága" -  Sugestão = "água"
Input = "luzs" -  Sugestão = "luzes"
Input = "prbabilidade" -  Sugestão = "probabilidade"

USO DO CORRETOR COM PALAVRA DIGITADA FALTANDO DUAS LETRAS
Input = "crrtor" -  Sugestão = "acrrtor"
Input = "prbbilidade" -  Sugestão = "aprbbilidade"

USO DO CORRETOR COM PALAVRA DIGITADA COM LETRAS ERRADAS
Input = "corrretor" -  Sugestão = "acorrretor"
Input = "probabillidade" -  Sugestão = "aprobabillidade"


O corretor está funcionando perfeitamente. Todavia, apenas para um tipo de erro: quando o usuário esquece alguma das letras da palavra.
A seguir, uma função de avaliação do modelo será implementada.

# Avaliação do modelo

## Carregar base de dados para teste do modelo

In [133]:
url = 'https://www.dropbox.com/s/nxquh7ozkojt7bb/palavras.txt?dl=1'

u_2 = urllib.request.urlopen(url)
data_2 = u_2.read()
u_2.close()

with open('palavras.txt', "wb") as f :
     f.write(data_2)

# lendo o arquivo
with open("palavras.txt", "r") as f:
     palavras = f.read()
  
print(palavras[:55])

podemos pyodemos
esse esje
já jrá
nosso nossov
são sãêo


Nesse momento, a base de dados contempla palavras corretas e palavras com erros ortográficos. Portanto, a próxima função será destinada à divisão destas categorias de palavras.

## Criação de função que divide base de dados para testar o modelo

In [134]:
def cria_dados_teste(nome_do_arquivo):
    lista_palavras_teste = []
    f = open(nome_do_arquivo, "r")
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

In [135]:
# teste da função cria_dados_teste
lista_teste = cria_dados_teste("palavras.txt")
lista_teste[:5]

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo')]

## Criação de função de avaliação do modelo

A função de avaliação será criada com o objetivo de mensurar o funcionamento do corretor. O *avaliador* recebe uma lista de palavras com erros, apresenta sugestões (por meio da função *corretor*) e retorna a média de acertos do modelo.

In [136]:
def avaliador(testes):
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
          acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f'Sugestões corretas: {taxa_acerto}% de {numero_palavras} palavras.')

Avaliando o estado atual do corretor:

In [137]:
avaliador(lista_teste)

Sugestões corretas: 2.05% de 195 palavras.


A seguir, uma função de correção de palavras digitadas com um caracter a mais será criada para melhorar a performance do correto.

# Criando função de correção mediante a remoção de uma letra

Para o processo de correção mediante a remoção de uma letra, um mecanismo de fatiamento da string de input, novamente será utilizado. Em seguida, a primeira letra do lado direito será removida. Por fim, as duas partes serão concatenadas e o resultado será sugerido ao usuário.



In [138]:
# exemplo do funcionamento de fatiamento e remoção de caracter
lista = "lóigica"
print(f'lista = {lista}')
print(f'1º remoção: {(lista[:0], lista[1:])}')
print(f'2º remoção: {(lista[:1], lista[2:])}')
print(f'3º remoção: {(lista[:2], lista[3:])}')
print(f'4º remoção: {(lista[:3], lista[4:])}')
print(f'5º remoção: {(lista[:4], lista[5:])}')
print(f'6º remoção: {(lista[:5], lista[6:])}')

lista = lóigica
1º remoção: ('', 'óigica')
2º remoção: ('l', 'igica')
3º remoção: ('ló', 'gica')
4º remoção: ('lói', 'ica')
5º remoção: ('lóig', 'ca')
6º remoção: ('lóigi', 'a')


Portanto, o segundo processo de correção atende o problema de palavras digitada equivocadamente com um caracter extra. Mais uma vez, o procedimento consiste em fatiar a palavra em duas partes de todas as formas demonstradas no exemplo. Em seguida, será removida a primeira letra da segunda parte de cada forma fatiada, com o objetivo de formar um escopo de palavras sugeridas ao usuário.

In [139]:
def deleta_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

Definida a função que realiza sugestão a partir da remoção de letras, ela será incorporada à uma atualização da função que gera palavras (*gerador_palavras*).

In [140]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letras(fatias)
    return palavras_geradas

In [141]:
# exemplo da função que corrige uma palavra com um caracter extra
corretor('llíngua')

'língua'

Uma vez implementada a segunda forma de correção, o modelo será avaliado novamente com a função *avaliador*.

In [142]:
avaliador(lista_teste)

Sugestões corretas: 41.03% de 195 palavras.


# Criando função de correção mediante a troca de uma letra


A função de troca de uma letra atuará para correção dos casos em que o usuário escreve a palavra com uma letra errada. O processo será iniciado novamente com o mecanismo de fatiamento da string. Em seguida, a primeira letra do lado direito será removida e outro caracter será adicionado em seu lugar. Após a inserção de todos os caracteres possíveis em todos os pontos de fatiamento, o corretor avaliará a palavra de maior ocorrência em seu corpus e o resultado será sugerido ao usuário.

In [143]:
def troca_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

Atualizando o modelo com a função de troca de letras:

In [144]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letras(fatias)
    palavras_geradas += troca_letras(fatias)
    return palavras_geradas

Testando a nova função do corretor com uma palavra errada:

In [145]:
corretor('langua')

'língua'

Nova avaliação do corretor que agora possui três funções (*insere, deleta e troca letras*).

In [146]:
avaliador(lista_teste)

Sugestões corretas: 73.33% de 195 palavras.


# Criando função de correção mediante a inversão de letras subsequentes


A função de troca de uma letra atuará para correção dos casos em que o usuário escreve a palavra invertendo a posição de duas letras. O processo será iniciado novamente com o mecanismo de fatiamento da string. Em seguida, a primeira letra do lado direito será invertida com a segunda. As partes serão concatenadas e geram uma palavra sugerida. Após a realização de todas as inversões possíveis, o corretor avaliará a palavra de maior ocorrência em seu corpus e o resultado será sugerido ao usuário.

In [147]:
def inverte_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
      if len(D) > 1:
         novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

Atualizando o modelo com a função que inverte letras:

In [148]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letras(fatias)
    palavras_geradas += troca_letras(fatias)
    palavras_geradas += inverte_letras(fatias)
    return palavras_geradas

Testando o corretor com uma palavra escrita com letras invertidas:

In [149]:
corretor('lnígua')

'língua'

Nova avaliação do corretor que agora possui quatro funções (*insere, deleta, troca e inverte letras*).

In [150]:
avaliador(lista_teste)

Sugestões corretas: 76.92% de 195 palavras.


#Erro do corretor e palavras desconhecidas
Uma vez que o corretor não possui uma taxa perfeita de acerto, podemos calcular quanto isso se deve às palavras não contempladas pelo corpus utilizado.

Portanto, a função avaliador será atualizada com a finalidade de informar a taxa de palavras desconhecidas presentes no corpus de teste. Para isso, a função deverá retornar a proporção de palavras que foram não corrigidas por não possuírem sua versão correta no corpus.

In [151]:
vocabulario = set(corpus_normalizado)

def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
          acertou += 1
        else:
            desconhecida += (correta not in vocabulario)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print(f'Sugestões corretas: {taxa_acerto}% de {numero_palavras} palavras.')
    print(f'Palavras desconhecidas: {taxa_desconhecidas}%')

In [152]:
avaliador(lista_teste, vocabulario)

Sugestões corretas: 76.92% de 195 palavras.
Palavras desconhecidas: 8.21%


# Expansão da correção
No estado atual, o corretor realiza apenas uma das ações para a correção das palavras (insere, deleta, troca e inverte letras). O próximo passo tem como objetivo que palavras sugeridas sejam geradas a partir da duas ações do corretor.

In [153]:
def gerador_expansao(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
    return novas_palavras

Verificando a capacidade de gerar uma palavra correta tendo dois erros de digitação:

In [154]:
# palavra com dois erros
palavra = "llínagua"

# aplicando o corretor com a expansão
palavras_expansao = gerador_expansao(gerador_palavras(palavra))

# verificando a presença da palavra correta na lista gerada
"língua" in palavras_expansao

True

A implementação extendida do corretor funcionou perfeitamente, porém é necessário verificar a performance desse modelo. Uma forma de fazer isso é contar o volume de palavras geradas a partir de uma entrada com dois erros:

In [155]:
len(palavras_expansao)

691744

Para otimizar a funcionalidade do corretor, será preciso diminuir o número de palavras avaliadas após a geração. Para isso, a função *novo_contador* irá separar entre aquelas geradas, pelo gerador simples e o expandido, as possíveis candidatas à sugestão. 

In [156]:
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavras_expansao = gerador_expansao(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_expansao)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
           candidatos.append(palavra)
    print(len(candidatos))
    palavra_correta = max(candidatos, key = probabilidade)
    return palavra_correta

A mesma entrada com dois erros foi submetida ao novo corretor. Nota-se que a quantidade de palavras avaliadas (*candidatos*) é significamente inferior quando comparada àquela gerada pelo processo expandido de correção:

In [157]:
novo_corretor(palavra)

2


'língua'

# Adaptação da função avaliador para palavras duas vezes distantes da correta

Com base nas funções *corretor* e *avaliador* serão criadas novas funções para atender a correção de palavras duas vezes distantes da correta.

In [158]:
# ajuste na função novo_corretor - retirar o print com o número de candidatos
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavras_expansao = gerador_expansao(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_expansao)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
           candidatos.append(palavra)
    palavra_correta = max(candidatos, key = probabilidade)
    return palavra_correta

# adaptação da função avaliador - palavras duas vezes distantes
def novo_avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        # ajuste - 'desconhecida' deve ter um valor fixo e não variável 
        desconhecida += (correta not in vocabulario) 
        if palavra_corrigida == correta:
          acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print(f'Sugestões corretas: {taxa_acerto}% de {numero_palavras} palavras.')
    print(f'Palavras desconhecidas: {taxa_desconhecidas}%')

Testando o corretor com a função que altera duas vezes a palavra errada:

In [159]:
novo_avaliador(lista_teste, vocabulario)

Sugestões corretas: 41.54% de 195 palavras.
Palavras desconhecidas: 8.21%


Para melhor compreensão do resultado gerado pela atualização da função *avaliador*, será realizada uma alteração em seu código. A nova função *analise_avaliador* retorna a concatenação de "palavra errada - palavra corrigida com o corretor antigo - palavra corrigida com o novo corretor".

In [160]:
def analise_novo_avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        desconhecida += (correta not in vocabulario) 
        if palavra_corrigida == correta:
          acertou += 1
        # análise do resultado (palavra com erro - antigo corretor - novo)
        else:
          print(errada + "-" + corretor(errada) + "-" + palavra_corrigida)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print(f'Sugestões corretas: {taxa_acerto}% de {numero_palavras} palavras.')
    print(f'Palavras desconhecidas: {taxa_desconhecidas}%')

Ao executar a função com a *lista_teste*, é possível analisar a performance dos dois corretores criados.

In [161]:
analise_novo_avaliador(lista_teste, vocabulario)

esje-este-está
jrá-já-há
nossov-nosso-posso
sãêo-são-não
dosa-dos-do
eme-ele-que
temfs-temos-tem
eàssa-essa-esta
sjava-java-estava
daõs-das-mas
céda-cada-da
noâ-no-o
fobma-forma-fora
enêão-então-não
èriar-criar-tirar
cóeigo-código-comigo
casío-caso-casa
tĩem-tem-em
dfados-dados-diabos
pgthon-python-pathan
nossah-nossa-nosso
asõim-assim-sim
teb-tem-de
atĩ-até-a
âem-em-de
foo-foi-o
empresà-empresa-emprego
serr-ser-se
entke-entre-então
méqodo-método-modo
ainàa-ainda-minha
van-vai-a
ûconteúdo-aûconteúdo-contendo
çeus-seus-eu
eû-eu-o
temeo-tempo-tem
semre-sempre-ser
elaá-ela-está
síó-só-se
prhojeto-projeto-prometo
siàe-site-se
seém-sem-se
peln-pelo-ele
aléra-aaléra-agora
tdia-dia-da
eẽsse-esse-disse
jé-é-o
nçosso-nosso-posso
sãô-são-não
odos-todos-do
siua-sua-seu
teos-temos-os
eũsa-essa-ela
jkva-java-nova
dms-dos-de
cava-casa-para
ános-nos-os
forûa-fora-fora
criôar-criar-cuidar
cóàigo-código-comigo
èaso-caso-isso
túem-tem-em
daáos-dados-vamos
nossk-nosso-nos
tãer-ter-tem
vté-até-é
búm-bem-u

# Análise dos corretores criados e conclusão

É possível perceber que a ocorrência de algumas palavras no corpus de treino influenciou a resposta do novo corretor para uma piora na performance. Por exemplo, a palavra errada 'fleiz' aparentemente foi um erro de digitação para 'feliz' com uma simples troca de letra. 

O corretor antigo sugeriu a palavra correta 'feliz'. 
O novo corretor, por sua vez, sugeriu 'fez', por ser uma palavra duas vezes distante de 'fleiz' que possui uma incidência maior no corpus de treino.

Podemos testar com outras palavras:

In [162]:
# palavra com erro de digitação para 'lógica'
palavra = 'lgica'
print(corretor(palavra))
print(novo_corretor(palavra))

lógica
fica


Comparação dos corretores criados:

In [163]:
print('Resultado do primeiro corretor:')
avaliador(lista_teste, vocabulario)

Resultado do primeiro corretor:
Sugestões corretas: 76.92% de 195 palavras.
Palavras desconhecidas: 8.21%


In [164]:
print('Resultado do segundo corretor:')
novo_avaliador(lista_teste, vocabulario)

Resultado do segundo corretor:
Sugestões corretas: 41.54% de 195 palavras.
Palavras desconhecidas: 8.21%


Apesar de ser uma função mais poderosa, o corretor antigo tem uma performance melhor para o presente corpus de treino e teste.